2장에서는 수집 단계에 유입되는 데이터를 알아보았다. 이번에는 수집 단계에서 나머지 스트리밍 파이프라인으로 데이터를 전달하는 부분을 알아보자.

# 메시지 큐 단계가 필요한 이유

데이터 수집에서 분석 단계로 바로 넘어가면 될 거 같은데, 메시지 큐 단계가 있는 이유가 뭘까?  

일반적으로 스티리밍 시스템은 여러 서버를 활용하는데, 서버들 간의 디커플링이 잘 되어 있어야 한다. 여타 분산 시스템처럼 스트리밍 시스템의 핵심도 시스템을 구성하는 수많은 서버 간의 통신이다. 여기에 수많은 통신 방식이 나오는데, **메시지 큐 모델(Message Queueing Model)** 을 선택하면 수집과 분석의 커플링르 끊고 통신을 수행할 수 있다. 이렇게 함으로써 디커플링을 만족시키고, 더 높은 수준에서 추상화된 시스템을 구현할 수 있다.

메시지 큐의 핵심 기능을 살펴보기 전에 메시지 큐 소프트웨어(레빗엠큐, ActieMQ, HornetQ, Kafka 등)들이 가진 특징과 이런 특징들이 어떻게 스트리밍 데이터 아키텍처에서 사용되는지 알아보자.

### producer, broker, consumer

메시지 큐는 Producer, Broker, Consumer라는 3가지 주요 요소가 있다. 여기서 Broker의 중요한 특징은 여러 개의 메시지 큐들이 모여서 운영된다는 점이다. 즉, 수집 단계에서 프로듀서가 메시지를 메시지 큐 단계로 보내면, 메시지 큐 단계에 있는 브로커가 받아서 분석 단계에 있는 컨슈머로 전달해준다. 

앞서 말했듯이 시스템에서 각 단계를 분리하는 것이 달성 목표 중 하나이다. 메시지 큐 단계를 적용하면 수집과 분석을 분리할 수 있다. 프로듀서가 데이터를 수집하는 속도에 비해 컨슈머가 데이터를 처리하는 속도가 느리면 중간에 데이터가 폭발해 유실될 수 있다. 그런데 이는 단순히 컨슈머를 늘린다고 해서 해결되지 않는다. 컨슈머는 언제든지 속도가 늦어질 수 있으며 특별한 사유로 연결이 끊어질 수도 있다. 만약 수집과 분석을 분리하면 이런 컨슈머와 브로커의 이슈가 생길 가능성을 낮출 수 있다. 이때 중요한게 메시지의 **지속적 저장(Durable Messaging)** 시스템이다.

수집 단계의 데이터 센터와 분석 단계의 데이터 센터가 다른 위치에 있다고 가정해보자. 그럼 어떠한 이유에서든(광섬유 공사 등) 두 센터 간의 통신이 며칠 간 끊길 수 있다. 데이터 유실이 있으면 안되는 사업을 하고 있다면, 데이터를 수집 단계에서 오래 저장할 수 있는 메시지 큐 기술을 적용해야 한다. 이 때 durable messaging 기능이 있다면 장애 복구가 가능하며, 일시적인 컨슈머 연결 종료에도 안전하게 데이터를 처리할 수 있다.  

한편, 메시지 큐 단계에서 분석 단계로 넘어간 이후로는 큐에서 메시지가 삭제되기 때문에 만약 과거 데이터를 확인하거나 분석을 해야 하는 서비스가 있다면, 메시지 큐 단계에서 오프라인 컨슈머들도 지원해야 한다. 오프라인 컨슈머는 시간 또는 일 단위로 데이터를 가져가서 처리하며, 처리가 완료된 과거 데이터 저장소로 데이터를 전달한다. 

### 메시지 전달 시맨틱

메시지 큐 소프트웨어에서 흔히 언급되는 메시지 전달 시맨틱은 다음과 같다.  

- At-most-once : 일부 메시지는 유실될 수 있으며, 유실되면 컨슈머에 도달하지 못한다.  
- At-least-once : 메시지가 절대 유실되지 않는다. 다만 컨슈머에서 동일 메시지를 여러 번 받을 수 있다.  
- Exactly-once : 모든 메시지가 유실되지 않고, 컨슈머도 반드시 1번의 메시지를 받는다.  
 
당연히 Exactly-once를 쓰고 싶다. 하지만 이 방법은 굉장히 어렵다.

아래는 각 부분이 갖고 있는 위험성이다.  

- 프로듀서 : 내부에서 메시지를 생성하고 네트워크 통신으로 브로커에 보내기 직전에 장애가 발생하면 메시지가 유실된다. 브로커에 메시지를 정상적으로 보냈지만 전달받았다는 응답을 받지 못하면 동일한 메시지가 브로커로 한번 더 전달도리 수 있다.  
- 프로듀서-브로커 간 네트워크 통신 : 통신에 이슈가 발생하면 프로듀서가 브로커로 메시지를 보내지 못한다. 또는 브로커가 메시지를 받아 저장했지만 저장 완료되었다는 응답을 보내지 못할 수도 있다.  
- 브로커 : 브로커에 장애가 발생하면 저장소에 저장하기 전 메모리가 가지고 있던 메시지가 유실된다. 또 컨슈머가 어떤 메시지까지 읽었는지 추적할 때 장애가 발생하면 동일한 메시지를 여러 번 처리할 수도 있다.  
- 메시지 큐 : 저장소에 장애가 발생하면 디스크에 저장된 메시지 일부가 유실된다.  
- 컨슈머-브로커 네트워크 통신 : 브로커가 컨슈머로 메시지를 보냈음에도 받지 못하거나, 응답을 받지 못해 여러 번 동일한 메시지를 받을 수 있다.  
- 컨슈머 : 동일한 메시지를 2번 이상 처리할 수 있다. 

복잡도가 낮으면 데이터 처리가 빠르지만, 그만큼 메시지가 중복되거나 유실될 가능성이 높아진다. 따라서 서비스를 만들 때는 타협점을 찾아야 한다. 예를 들어 웹 기반 스티리밍 데이터 분석 제품은 일부 메시지가 유실되더라도 신뢰도에 영향을 끼치지 않을 수 있지만, 사기 결제 검출 시스템에서는 하나의 메시지 유실이 제품 신뢰도에 큰 영향을 미칠 수 있다.  

안타깝게도 Kafka, ActiveMQ 등 메시지 시스템은 Exactly-once 시스템을 지원하지 않는다. 대신 프로듀서-컨슈머 통신에서 exactly-once를 할 수 있도록 메타데이터를 제공한다. 그러기 위해 필요한 조건은 아래 두가지이다.  
- 프로듀서가 브로커로 메시지를 보낼 때, 각 메시지들을 추적해야 한다. 장애가 발생하면 플듀서가 이전에 보낸 데이터를 정상적으로 받았는지 브로커로 요청하고, 확인응답(Acknowledgement)를 받는 식으로 추적이 이루어져야 한다.  
- 읽은 메시지의 메타데이터를 저장해야 한다. 메시징 시스템에 따라 메타데이터의 종류는 다른데, 메타데이터를 저장할 때 장애가 발생하는 경우에 주의해야 한다.  

이러면 프로듀서가 보낸 메시지가 지속적으로 추적되고 컨슈머에서는 메타데이터를 토대로 메시지를 처리하니까 메시지가 처리되는 영역의 최대, 최소, 평균 시간을 알아낼 수 있다. 그리고 프로듀서가 브로커로 메시지를 보낼 때 반드시 한 번만 보내게 구성했고, 컨슈머는 메타데이터를 통해 이미 처리된 메시지인지 체크한다.  

또한 컨슈머는 메시지 시스템에 관련된 메타데이터를 추적할 뿐만 아니라 메시지에서 처리해야 하는 내부 페이로드 데이터를 확인할 때도 사용된다. 따라서 exactly-once를 달성하면서 메시지 처리 품질과 중복 탐지도 달성할 수 있다.

# 보안

지금까지 브로커나 컨슈머의 장애에 대응해 데이터를 복구하는 방법을 알아보았다. 하지만 데이터가 전달되고 있을 때나 저장되고 있을 때 안전하게 보호되며, 데이터 처리에 이슈가 없어야 한다. 브로커들 간에 인증할 수 있는지, 프로듀서는 인증할 수 있는지, 인가된 프로듀서만 메시지를 전달할 수 있는지, 통신간 메시지를 암호화할 수 있는지, 저장된 메시지를 암호화할 수 있는지 등 고려해야 할 보안 사항이 많다. 메시지 큐 소프트웨어가 보안 기능도 제공하는 것은 아니라, 사내 보안팀과 협업해야 한다.

# 장애 허용(Fault tolerance)

분산 시스템에서는 장애가 날 것에 관심을 두어야 한다.통신 장애는 충분히 발생할 수 있기 때문에 브로커에 내구성 있는 로컬 저장소를 두기도 하고, 메시지 큐 소프트웨어가 내구 저장소 저장 기능을 지원하지 않는 경우에는 메시지 유실을 막기 위한 다른 방법을 찾아야 한다.  

- **로커에 장애가 발생**  
Durable storage를 사용한다고 가정했을 때 브로커가 저장소에 저장하기 전에 메모리가 가지고 있던 메시지를 어떻게 처리할지를 고민하면 된다. 해결법으로는 (1) 프로듀서가 브로커로 메시지를 보낸 다음, 디스크에 안전하게 저장되었다는 응답을 받거나 (2) 메시지 큐 소프트웨어가 2개 이상의 브로커로 메시지를 복제해 해결하거나 (3) 브로커가 메시지를 처리할 때 메모리가 메시지를 최대한 작게 가지고 있는 방법이다.  
- **서로 다른 브로커 사이의 네트워크 장애 발생**  
브로커 사이에 이슈가 발생하면 (1) 새롭게 메시지를 복제하기 위해 어떤 브로커를 선택할지 (2) 네트워크 통신이 장애 복구되면 어떻게 동작할지 (3) 네트워크 장애로 판단하기 위해 지연 정도를 설정할 수 있는지 (4) 네트워크가 장애 이후에 복구되지 못할 때 어떻게 동작할지 (5) 프로듀서가 클러스터에서 탈락된 브로커에 메시지를 보내면 데이터가 어떻게 될지 등을 고려해보아야 한다.  
- **저장소에 장애 발생**  
저장소 장에 엄청난 데이터가 유실될 수 있지만, 디스크 장애는 흔히 발생하는 일이고 운영자들은 이에 대응하는데 익숙하다. 여러 브로커를 지원하는 메시지 큐 소프트웨어는 다음과 같은 질문을 생각해보아야 한다. (1) 유실된 데이터에 대한 복제본이 있는지 (2) 복제가 진행되고 있을 때 디스크에 기록되지 않은 상태에서 장애가 발생하면 데이터가 영원히 유실되는지 (3) 브로커를 복구하는 방법은 어떻게 되는지

지금까지 메시지 큐 단계에 대해 생각할 때 명심해야 할 핵심 개념을 다루었다.